# Train for train.py

Jupyter Cell by cell from the train.py script

In [1]:
import logging
import datetime

import torchvision
from tqdm.notebook import tqdm
import numpy as np
from dataset import PinesDataset
from torch.utils.data import random_split
import torch
from torch.utils.tensorboard import SummaryWriter

import dataset
import models
from train import train_one_epoch, evaluate, load_hyperparameters

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)


In [2]:
hyp = load_hyperparameters("cfg/hyperparameters.json")

In [2]:
writer = SummaryWriter("../../runs/pines")

In [3]:
# device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Method 1 : Loading data from memory 
pines_dataset = PinesDataset("/mnt/sda1/1-Pins/DATASET/", scale=hyp.scale)
train_set, val_set, test_set = random_split(
    dataset=pines_dataset,
    lengths=[0.8, 0.1, 0.1],
    generator=torch.Generator().manual_seed(42),
)

INFO-20240306-10:55:33|775 images and targets loaded


ou choisir la méthode 2

In [4]:
# Method 2 : loading data from memory and save it as it will be loaded after instead of all dataset

dataset.save_train_val_test_sets(root="/mnt/sda1/1-Pins/DATASET/", scale=1, save_dir="../../datasets/")
# train_set = torch.load("../datasets/train_set.pt")
# val_set = torch.load("../datasets/val_set.pt")
# test_set = torch.load("../datasets/test_set.pt")


INFO-20240306-11:43:58|8452 images and targets loaded


Dataldoaders

In [5]:
dataloaders = dataset.DataLoaders(
    train=torch.utils.data.DataLoader(
        train_set, batch_size=hyp.batch_size, collate_fn=dataset.collate_fn
    ),
    validation=torch.utils.data.DataLoader(
        val_set, batch_size=hyp.batch_size, collate_fn=dataset.collate_fn
    ),
    test=torch.utils.data.DataLoader(
        test_set, batch_size=hyp.batch_size, collate_fn=dataset.collate_fn
    ),
)


In [6]:
# Example code for plotting images batch on tensorboard
example_images, example_targets = dataset.prepare_batch(next(iter(dataloaders.train)), device)

img_grid = torchvision.utils.make_grid(example_images)
writer.add_image('pines_images', img_grid)


In [7]:
model = models.get_fasterrcnn_mobilenet_v3().to(device)
#     weights_path="../pretrained-models/fasterrcnn-scale1-bs32-epochs100-weights.pth"
# ).to(device)


INFO-20240306-10:55:51|FasterRCNN loaded with no pretrained weights


In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=hyp.learning_rate, momentum=0.9, weight_decay=0.0005)


Train the model

In [12]:
for epoch in range(hyp.num_epochs):
    train_metric_logger = train_one_epoch(model, optimizer, dataloaders.train, device, epoch, print_freq=1)
    evaluator, eval_metric_logger = evaluate(model, dataloaders.validation, device, epoch, print_freq=1)
    writer.add_scalars(
        'losses', 
        {
            'train_sum':train_metric_logger.loss_sum.avg,
            'train_box_reg':train_metric_logger.meters["loss_box_reg"].avg,
        }, 
        epoch
    )
    writer.add_scalar('mAP', np.mean(evaluator.coco_eval.eval['precision'][:,:,:,0,2]), epoch)

    # TODO: plot the mAP for evaluator
    # TODO: give metric loggers and evaluator to a tensorboard handler for plotting the metrics

    # Track best performance, and save the model's state
    # if avg_vloss < best_vloss:
    #     best_vloss = avg_vloss
    #     model_path = 'model_{}_{}'.format(timestamp, epoch_number)
    #     torch.save(model.state_dict(), model_path)


print("Finished Training")
torch.cuda.empty_cache()


Train: [1]	[ 1/60]	eta: 0:03:01	loss_classifier: 0.0828 (0.0828)	loss_box_reg: 0.2574 (0.2574)	loss_objectness: 0.0033 (0.0033)	loss_rpn_box_reg: 0.0026 (0.0026)	loss_sum: 0.3461 (0.3461)	time: 3.0249	data: 0.0019	max mem: 5756
Train: [1]	[ 2/60]	eta: 0:01:50	loss_classifier: 0.0828 (0.0932)	loss_box_reg: 0.2574 (0.2857)	loss_objectness: 0.0033 (0.0043)	loss_rpn_box_reg: 0.0026 (0.0028)	loss_sum: 0.3461 (0.3860)	time: 1.8808	data: 0.0016	max mem: 5870
Train: [1]	[ 3/60]	eta: 0:01:25	loss_classifier: 0.1035 (0.1044)	loss_box_reg: 0.2782 (0.2832)	loss_objectness: 0.0053 (0.0047)	loss_rpn_box_reg: 0.0029 (0.0032)	loss_sum: 0.4141 (0.3954)	time: 1.4721	data: 0.0014	max mem: 5870
Train: [1]	[ 4/60]	eta: 0:01:13	loss_classifier: 0.1035 (0.1134)	loss_box_reg: 0.2782 (0.3047)	loss_objectness: 0.0053 (0.0053)	loss_rpn_box_reg: 0.0029 (0.0033)	loss_sum: 0.4141 (0.4267)	time: 1.2823	data: 0.0014	max mem: 5870
Train: [1]	[ 5/60]	eta: 0:01:04	loss_classifier: 0.1044 (0.1116)	loss_box_reg: 0.2810 (0

In [6]:
now = datetime.datetime.now().strftime("%Y%m%d_%H:%M:%S")
model_weights = f"../../../pretrained-models/fasterrcnn-scale{hyp.scale}-bs{hyp.batch_size}-epochs{hyp.num_epochs}-weights-{now}.pth"
print(model_weights)
# torch.save(model.state_dict(), model_weights)

../../../pretrained-models/fasterrcnn-scale0.1-bs32-epochs2-weights-20240306_12:07:06.pth


In [8]:
import os
output_dir_now = os.path.join("../../outputs/", now)
os.makedirs(output_dir_now, exist_ok=True)


In [ ]:
writer.flush()
writer.close()

import shutil
shutil.rmtree(log_dir)

In [ ]:
import matplotlib.pyplot as plt
from torchvision.utils import draw_bounding_boxes

In [ ]:
images_batch, targets_batch = next(iter(dataloaders["test"]))
images = [image for image in images_batch]
targets = [{k: v for k, v in t.items()} for t in targets_batch]


In [ ]:
img = images[0]
img_int = torch.tensor(img*255, dtype=torch.uint8)
with torch.no_grad():
    prediction = model([img.to(device)])
    pred = prediction[0]

In [ ]:
print(f"Boxes:\n\t{pred['boxes']}\nScores:\n\t{pred['scores']}")

In [ ]:
figure = plt.figure(figsize=(14, 8*32))
cols, rows = 2, 32


for i, img in enumerate(images):
    img_int = torch.tensor(img*255, dtype=torch.uint8)
    with torch.no_grad():
        prediction = model([img.to(device)])
        pred = prediction[0]
    figure.add_subplot(rows, cols, i*2+1)
    plt.imshow(
        draw_bounding_boxes(
            image=img_int,
            boxes=pred['boxes'],
            colors=["red"]*len(pred['boxes']),
            width=1
    ).permute(1, 2, 0))
    
    figure.add_subplot(rows, cols, i*2+2)
    plt.imshow(
        draw_bounding_boxes(
            image=img_int,
            boxes=targets[i]["boxes"],
            colors=["blue"]*len(targets[i]['boxes']),
            width=1
    ).permute(1, 2, 0))
plt.show()

